In [7]:
pwd

'/Users/Frost/Desktop/VŠE/5. Semestr/BP/vizualizace'

In [8]:
import pandas as pd
# Assuming you have your data stored in a CSV file named 'data.csv'
df = pd.read_csv('healthcare_dataset2.csv', sep=';')

In [18]:
%%time
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook
output_notebook()
# Count the occurrences of each blood type
blood_type_counts = df['Blood Type'].value_counts()
# Convert the counts to a DataFrame
data = pd.DataFrame({'blood_type': blood_type_counts.index, 'count': blood_type_counts.values})
# Create a ColumnDataSource
source = ColumnDataSource(data)
# Create a figure
p = figure(x_range=data['blood_type'], height=350, title='Number of Each Blood Type',
           toolbar_location=None, tools="")
# Create a bar plot
p.vbar(x='blood_type', top='count', width=0.9, source=source, color="skyblue")
# Customize the plot
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.axis_label = 'Blood Type'
p.yaxis.axis_label = 'Count'
# Show the plot
show(p)


Loading BokehJS ...

CPU times: user 171 ms, sys: 9.67 ms, total: 180 ms
Wall time: 285 ms


In [10]:
%%time
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

# Enable Bokeh to output to Jupyter Notebook
output_notebook()

# Create a histogram of ages
p = figure(title="Histogram of Ages", x_axis_label="Age", y_axis_label="Frequency", width=600, height=400)

# Create the histogram
hist, edges = np.histogram(df['Age'], bins=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="skyblue", line_color="black")

# Show the plot
show(p)


Loading BokehJS ...

CPU times: user 89.7 ms, sys: 5.62 ms, total: 95.3 ms
Wall time: 175 ms


In [11]:
%%time
import numpy as np
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Whisker
from bokeh.io import output_notebook
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral11

output_notebook()

# Group data by Medical Condition
grouped = df.groupby('Medical Condition')['Days in Hospital']

# Calculate quantiles
qs = grouped.quantile([0.25, 0.5, 0.75])
qs = qs.unstack().reset_index()
qs.columns = ['Medical Condition', 'q1', 'q2', 'q3']
df_new = pd.merge(df, qs, on='Medical Condition', how='left')

# Calculate IQR outlier bounds
iqr = df_new['q3'] - df_new['q1']

mad = (df_new['Days in Hospital'] - df_new['Days in Hospital'].mean()).abs().mean()

df_new['upper'] = df_new['q3'] + 1 * mad
df_new['lower'] = df_new['q1'] - 1 * mad

source = ColumnDataSource(df_new)

# Create a figure
p = figure(x_range=df_new['Medical Condition'].unique(), tools="", toolbar_location=None,
           y_range=(0, 35),
           title="Days in Hospital by Medical Condition",
           background_fill_color="#eaefef", y_axis_label="Days in Hospital",
           width=800, height=400)

# Whisker for outlier range
whisker = Whisker(base="Medical Condition", upper="upper", lower="lower", source=source)
whisker.upper_head.size = whisker.lower_head.size = 15
p.add_layout(whisker)

# Quantile boxes
cmap = factor_cmap("Medical Condition", Spectral11, df_new['Medical Condition'].unique())
p.vbar("Medical Condition", 0.7, "q2", "q3", source=source, color=cmap, line_color="black")
p.vbar("Medical Condition", 0.7, "q1", "q2", source=source, color=cmap, line_color="black")

# Outliers
outliers = df[~df_new['Days in Hospital'].between(df_new['lower'], df_new['upper'])]
p.scatter("Medical Condition", "Days in Hospital", source=outliers, size=6, color="black", alpha=0.3)

p.xgrid.grid_line_color = None
p.axis.major_label_text_font_size = "14px"
p.axis.axis_label_text_font_size = "12px"

show(p)

Loading BokehJS ...

CPU times: user 537 ms, sys: 31.5 ms, total: 569 ms
Wall time: 790 ms
